In [2]:
from keras.models import Sequential

Using TensorFlow backend.


model=Sequential()  
model.add(Embedding(voc,hidden_size,input_length))  
model.add(Dense(2, input_dim=4, init='uniform',activaition='linear'))  
model=Sequential()  
model.add(Dense(4, input_dim=10, init='normal',activation='relu'))  
model.add(Dense(1, activation='sigmoid'))  
model.summary() : 요약정보  
model.compile() : 함수설정  
model.fit() : 학습  
model.evaluate() : 모델 평가  
model.predict() : 예측  
model.save(), load_model(): 모델 저장 & 불러오기  

In [8]:
import tensorflow as tf
tf.__version__ # 텐서플로우 버전
tf.get_default_graph() # 기본 그래프 정보 

In [24]:
testGraph=tf.Graph() #사용자 그래프 생성
with testGraph.as_default(): # 사용자 그래프에 노드 정의
    x=tf.constant(10)
    y=x+20

In [23]:
# Session():세선객체생성, 분산처리환경 노드를 연결
# run():연산 수행 -> 결과를 리턴
# close():세선 종료
x=tf.constant(5)
y=x+10
sess=tf.Session()
print(sess.run(x))
print(sess.run(y))
sess.close()

5
15


In [21]:
with tf.Session() as sess: # 특별히 그래프를 지정하지 않았으므로, 기본 그래프를 사용해서 세션을 실행
    print(sess.run(x))

5


In [25]:
with tf.Session(graph=testGraph) as sess: 
    print(sess.run(x))

10


In [28]:
# sess=tf.InteractiveSession()
# x.eval()
# y.eval()

In [29]:
var1=tf.Variable(5, name="var1")
var2=tf.add(var1,5, name="var2")
var1.op.name

'var1'

In [34]:
# 이름공간(name space, 폴더와 비슷)을 이용한 노드 이름 중복 방지
# tf.name_space():이름 공간 생성
with tf.name_scope("ns1"):
    v3=tf.Variable(1, name="var3")
    v4=tf.add(v3,2,name="var4")
print(v3.op.name)

ns1_3/var3


In [145]:
# 입력 4개 -> 출력 2개 (hidden_size)

# "hihello"
# "hihell" => 입력
# "ihello" => 출력
# time_step:6, input_dim:5(전체 vocab(hihello로 만들어진) 사이즈), hidden_size:5

idx2char=["h","i","e","l","o"] # hihello에 set 함수를 쓰면 이렇게 나옴
# 원래는 set 함수를 활용해 분리 후 원핫인코딩(to_categorical)으로 표현해야함
xdata=[[0,1,0,2,3,3]]# xdata="hihell"를 인덱스로 표현
x_onehot=[[
    [1,0,0,0,0],
    [0,1,0,0,0],
    [1,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,1,0]]]
ydata=[[1,0,2,3,3,4]] # "ihello"

tf.reset_default_graph()

num_classes=5
input_dim=5 # 입력데이터의 차원(단어벡터의 개수)
hidden_size=5
batch_size=1
sequence_lentgh=6 #hihell까지 입력되므로 6글자
learning_rate=0.1

x=tf.placeholder(tf.float32, [None,sequence_lentgh, input_dim])
y=tf.placeholder(tf.int32, [None,sequence_lentgh])


#### 일반적으로 고정되는 코드들
cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)  # num_units는 출력개수
ini_state = cell.zero_state(batch_size, tf.float32) # 가중치 초기상태 설정
outputs, _states = tf.nn.dynamic_rnn(cell, x, initial_state = ini_state, dtype = tf.float32)
# outputs : 위로 올라가는 것, states : 현재 상태

x_fc=tf.reshape(outputs,[-1, hidden_size])
outputs=tf.contrib.layers.fully_connected(inputs=x_fc, num_outputs=num_classes)
outputs=tf.reshape(outputs,[batch_size, sequence_lentgh, num_classes])
#                          [1, 6, 5]=>[[[0.1,0.2,0.5,0.1,0.1]],[],[]]

weights=tf.ones([batch_size, sequence_lentgh])
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs,targets=y, weights=weights)
loss=tf.reduce_mean(sequence_loss)

prediction= tf.argmax(outputs, axis=2) # outputs에서 prediction은 num_classes로 axis=2로 줘야함

train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(101):
        lv, _=sess.run([loss, train], 
                       feed_dict={x:x_onehot, y:ydata})
        res=sess.run(prediction, feed_dict={x:x_onehot})
        print("step:", i, "cost:",lv, "prediction:",res,
              "true Y:", ydata)
        # prediction:[[1 0 2 2 3 1]] true Y:[1 0 2 3 3 4]
        # .....
        # prediction:[[1 0 2 3 3 4]] true Y:[1 0 2 3 3 4]
        resStr=[idx2char[c] for c in np.squeeze(res)]
        print("predction str:","".join(resStr))
        
        
# 학습 hihell -> ihello를 출력하고 싶음
# 훈련과정 
# hihell -> elhill 
# ....
# hihell -> ihehho
# ....
# hihell -> ihello (many to many)
# 만약 many to one으로 출력하면 hihell -> o 이렇게 출력됨

step: 0 cost: 1.6112338 prediction: [[2 2 2 1 1 1]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeiii
step: 1 cost: 1.6168083 prediction: [[2 2 2 2 2 2]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeee
step: 2 cost: 1.6073672 prediction: [[2 2 2 2 2 2]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeee
step: 3 cost: 1.6017066 prediction: [[2 2 2 2 2 0]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeeh
step: 4 cost: 1.586414 prediction: [[2 2 2 2 0 0]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeehh
step: 5 cost: 1.5756149 prediction: [[2 2 2 0 0 0]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeehhh
step: 6 cost: 1.5621663 prediction: [[2 2 2 2 0 0]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeehh
step: 7 cost: 1.544249 prediction: [[2 2 2 2 0 0]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeehh
step: 8 cost: 1.5267744 prediction: [[2 2 2 2 0 0]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeehh
step: 9 cost: 1.508956 prediction: [[2 2 2 0 0 0]] true Y: [[1, 0, 2, 3, 3,

In [146]:
idx2char=['h', 'i', 'e', 'l', 'o']
xdata=[[0,1,0,2,3,3]] # xdata="hihell"
                     #        "ihello"
        #   cell -> ihello
x_onehot=[[
     [1,0,0,0,0],
     [0,1,0,0,0],
     [1,0,0,0,0],
     [0,0,1,0,0],
     [0,0,0,1,0],
     [0,0,0,1,0]]]
ydata=[[1,0,2,3,3,4]] #"ihello"

tf.reset_default_graph()

num_classes=5
input_dim=5 #입력데이터의 차원(단어벡터의 길이)
hidden_size=5
batch_size=1
sequence_length=6 #hihell
learning_rate=0.1

x=tf.placeholder(tf.float32, [None,sequence_length,input_dim])
y=tf.placeholder(tf.int32, [None,sequence_length])

cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
ini_state=cell.zero_state(batch_size, tf.float32)
outputs, _states=tf.nn.dynamic_rnn(cell, x, initial_state=ini_state, dtype=tf.float32)

x_fc=tf.reshape(outputs, [-1, hidden_size])
outputs=tf.contrib.layers.fully_connected(inputs=x_fc, num_outputs=num_classes)
outputs=tf.reshape(outputs, [batch_size,sequence_length,num_classes])
#                           [1, 6, 5] =>[[[0.1,0.2,0.5,0.1,0.1] ],[ ],...,[ ]]

weights=tf.ones([batch_size, sequence_length])
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs,targets=y, weights=weights)
loss=tf.reduce_mean(sequence_loss)

prediction=  tf.argmax(outputs,axis=2)

train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(101):
        lv, _=sess.run([loss, train], 
                       feed_dict={x:x_onehot, y:ydata})
        res=sess.run(prediction,feed_dict={x:x_onehot})
        print("step:",i,"cost:",lv, "prediction:", res, 
              "true Y:", ydata)
        resStr=[idx2char[c] for c in np.squeeze(res)]
        print("predction str:","".join(resStr))

step: 0 cost: 1.6160024 prediction: [[4 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: oeeeoo
step: 1 cost: 1.576425 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 2 cost: 1.542451 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 3 cost: 1.4947838 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 4 cost: 1.445278 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 5 cost: 1.4093639 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 6 cost: 1.374883 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 7 cost: 1.3337227 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 8 cost: 1.3077408 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
predction str: eeeeoo
step: 9 cost: 1.2668854 prediction: [[2 2 2 0 4 4]] true Y: [[1, 0, 2, 3, 3, 

In [147]:
test=[[3,1,0,2,1]]
# np.squeeze(test) # 차원 내려주는 함수
test2=[idx2char [c] for c in np.squeeze(test)]
test2

['l', 'i', 'h', 'e', 'i']

In [176]:
import numpy as np
from tensorflow.contrib import rnn
import pprint 
tf.reset_default_graph() # session 만들기 전에 그래프 초기화하기
pp=pprint.PrettyPrinter(indent=4)
sess=tf.InteractiveSession()

In [164]:
h=[1,0,0,0] # hello => 4글자
e=[0,1,0,0]
l=[0,0,1,0]
o=[0,0,0,1]

In [150]:
with tf.variable_scope('first_cell') as scope:
    hidden_size=2 # 2차원
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)
    xdata=np.array([[h]], dtype=np.float32)
    outputs,_states=tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    pp.pprint(_states.eval())

2 2
array([[[0.680201  , 0.21927692]]], dtype=float32)
array([[0.680201  , 0.21927692]], dtype=float32)


In [160]:
with tf.variable_scope('sec_cell') as scope:
    hidden_size=6 # 6차원 , 추출하고자 하는 특징
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o]], dtype=np.float32)
    print(xdata.shape) # (1, 5, 4) =(batch_size,seq_length,input_dim)
    pp.pprint(xdata)
    outputs,_states=tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.4018405 ,  0.23028307,  0.381782  , -0.46824923,
          0.12643304,  0.462466  ],
        [-0.18211307,  0.33006454,  0.6119752 , -0.7052023 ,
          0.42372483, -0.57520837],
        [ 0.5645258 ,  0.736937  ,  0.22579458, -0.41114545,
         -0.6542987 , -0.62958586],
        [ 0.61032426,  0.29288793, -0.38336554,  0.19066586,
         -0.667347  , -0.72544116],
        [ 0.5200934 , -0.529092  , -0.81993884,  0.19085152,
         -0.39565524,  0.12361063]]], dtype=float32)


In [165]:
with tf.variable_scope('thr_cell') as scope:
    hidden_size=2 # 2차원 , 추출하고자 하는 특징
    cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    print(xdata.shape) # (3, 5, 4) =(batch_size,seq_length,input_dim)
    pp.pprint(xdata)
    outputs,_states=tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.0197665 ,  0.5191908 ],
        [-0.652543  , -0.2532097 ],
        [ 0.27579546, -0.38055763],
        [ 0.16919109, -0.47628978],
        [ 0.0380247 ,  0.17428748]],

       [[-0.33075398,  0.17651384],
        [-0.36869687, -0.24236642],
        [ 0.204504  , -0.44471413],
        [ 0.23633698, -0.41989827],
        [ 0.20963773, -0.44295737]],

       [[-0.08040207, -0.646203  ],
        [ 0.44242695, -0.20805171],
        [-0.268038  ,  0.24035051],
        [-0.44657096,  0.04269702],
        [-0.01113443, -0.6023232 ]]], dtype=float32)


In [166]:
# M(시트),V(그래프),C(컨트롤러(막대)) 분리
with tf.variable_scope('lstm_thr_cell') as scope:
    hidden_size=2 # 2차원 , 추출하고자 하는 특징
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    print(xdata.shape) # (1, 5, 4) =(batch_size,seq_length,input_dim)
    pp.pprint(xdata)
    outputs,_states=tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.00645857,  0.06096779],
        [ 0.14418894,  0.0480664 ],
        [ 0.11326347, -0.17120096],
        [ 0.15688924, -0.33632037],
        [ 0.22947495, -0.35370985]],

       [[ 0.15885071,  0.00111706],
        [ 0.17092356, -0.15006763],
        [ 0.19864793, -0.35254478],
        [ 0.2301636 , -0.46135247],
        [ 0.2575241 , -0.525177  ]],

       [[ 0.05847339, -0.19343565],
        [ 0.12273372, -0.34363112],
        [ 0.31122842, -0.24834503],
        [ 0.3651099 , -0.22102582],
        [ 0.24124447, -0.40783525]]], dtype=float32)


In [167]:
with tf.variable_scope('lstm_thr_cell_2') as scope:
    hidden_size=2 # 2차원 , 추출하고자 하는 특징
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    print(xdata.shape) # (1, 5, 4) =(batch_size,seq_length,input_dim)
    pp.pprint(xdata)
    outputs,_states=tf.nn.dynamic_rnn(cell, xdata, sequence_length=[5,3,4],dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.0096851 ,  0.13529623],
        [ 0.11435476,  0.03642866],
        [ 0.06805395,  0.06833284],
        [ 0.00590002,  0.09689396],
        [ 0.12565891, -0.02646654]],

       [[ 0.11113451, -0.06463631],
        [ 0.19101495, -0.13958214],
        [ 0.12195124, -0.11071184],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.0471239 ,  0.0449914 ],
        [-0.08238813,  0.08419165],
        [ 0.07238495,  0.00109478],
        [ 0.15266635, -0.06875317],
        [ 0.        ,  0.        ]]], dtype=float32)


In [168]:
# weight 초기화
with tf.variable_scope('init_lstm_thr_cell') as scope:
    hidden_size=2 # 2차원 , 추출하고자 하는 특징
    batch_size=3 # 3개 단위로 학습. 몇번 업데이트 할지
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,l],
                   [l,l,e,e,l]], dtype=np.float32)
    print(xdata.shape) # (1, 5, 4) =(batch_size,seq_length,input_dim)
    pp.pprint(xdata)
    init_state=cell.zero_state(batch_size, tf.float32)
    outputs,_states=tf.nn.dynamic_rnn(cell, xdata, initial_state=init_state,dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    
    #데이터 : 100개
        #1에폭 : 100개 데이터를 학습
    # batch_size:20개, batch_length:100/20=5번
        #(20개 단위로 학습(forward, backward))

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.02525274,  0.07550837],
        [-0.13323417, -0.00758323],
        [ 0.02142371, -0.01214548],
        [ 0.1080116 , -0.00365101],
        [ 0.12789305, -0.09502799]],

       [[-0.09014505, -0.03834355],
        [-0.01866181, -0.11696686],
        [ 0.08297458, -0.07268869],
        [ 0.15771493, -0.03637911],
        [ 0.20971847, -0.00682593]],

       [[ 0.09065913,  0.00198483],
        [ 0.15849572,  0.0109465 ],
        [ 0.14155647, -0.01996978],
        [ 0.05042074, -0.04210166],
        [ 0.11948559, -0.01452091]]], dtype=float32)


In [173]:
batch_size=3 
seq_len=5 #입력에 해당하는 글자의 수
in_dim=3 # 각 문자의 shape 3
xdata=np.arange(45).reshape(batch_size, seq_len, in_dim) #(3,5,3)
print(xdata) # [0,1,2],[3,4,5],... 리스트 하나하나가 in_dim을 의미

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]]

 [[15 16 17]
  [18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]]

 [[30 31 32]
  [33 34 35]
  [36 37 38]
  [39 40 41]
  [42 43 44]]]


In [180]:
## 에러 발생
with tf.variable_scope('multi_rnn_cell') as scope:
    
    cell=tf.contrib.rnn.BasicLSTMCell(num_units=5) 
    cell=tf.contrib.rnn.MultiRNNCell([cell]*3) #layer 3개 
    outputs,_states=tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

In [181]:
## 에러 발생
# rnn에서 cost를 구하는 과정
ydata=tf.constant([[1,1,1]])
#        원래 값 : [[[0,1],[0,1],[0,1]]]
#     소프트맥스에 넣으면 최대값 인덱스만 출력되서 [[1,1,1]] 출력

prediction1=tf.constant([[[0.3,0.7],[0.1,0.9],[0.3,0.7]]]) # hidden_size=2, sequence_lentgh=3
prediction2=tf.constant([[[0.4,0.6],[0.4,0.6],[0.6,0.4]]])

loss1=tf.contrib.seq2seq.sequence_loss(prediction1, ydata)
sess.run(tf.global_variables_initializer())
print(loss1.eval())